In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

#データ読み込み
train = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")
gender_submission = pd.read_csv("dataset/gender_submission.csv")

#特徴量エンジニアリング
#train
t_train = train["Survived"]
train["FamilySize"] = train["SibSp"] + train["Parch"] 
x_train = train[["Pclass","Sex","Age","FamilySize","Fare","Embarked"]]
x_train["Sex"].replace(["male","female"],[0,1],inplace=True)
# Embarkedをonehot変換
Embarked = pd.get_dummies(x_train["Embarked"])
Embarked.replace([False,True],[0,1],inplace=True)
x_train = x_train.drop(columns=['Embarked'])
x_train = pd.concat([x_train,Embarked],axis = 1)
x_train.fillna(x_train.mean(), inplace=True)
#正規化
x_train = (x_train - x_train.mean()) / x_train.std()
x_train
#test
test["FamilySize"] = test["SibSp"] + test["Parch"] 
x_test = test[["Pclass","Sex","Age","FamilySize","Fare","Embarked"]]
x_test["Sex"].replace(["male","female"],[0,1],inplace=True)
# Embarkedをonehot変換
Embarked = pd.get_dummies(x_test["Embarked"])
Embarked.replace([False,True],[0,1],inplace=True)
x_test = x_test.drop(columns=['Embarked'])
x_test = pd.concat([x_test,Embarked],axis = 1)
x_test.fillna(x_test.mean(), inplace=True)
#正規化
x_test = (x_test - x_test.mean()) / x_test.std()
x_test

C:\Users\logihaal\AppData\Local\Temp\ipykernel_24536\1753555050.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x_train["Sex"].replace(["male","female"],[0,1],inplace=True)
C:\Users\logihaal\AppData\Local\Temp\ipykernel_24536\1753555050.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  x_tra

,Pclass,Sex,Age,FamilySize,Fare,C,Q,S
0,0.872436,-0.755024,0.334592,-0.552780,-0.497811,-0.567462,2.840354,-1.349059
1,0.872436,1.321292,1.323944,0.105516,-0.512660,-0.567462,-0.351227,0.739484
2,-0.315441,-0.755024,2.511166,-0.552780,-0.464532,-0.567462,2.840354,-1.349059
3,0.872436,-0.755024,-0.259019,-0.552780,-0.482888,-0.567462,-0.351227,0.739484
4,0.872436,1.321292,-0.654760,0.763813,-0.417971,-0.567462,-0.351227,0.739484
...,...,...,...,...,...,...,...,...
413,0.872436,-0.755024,0.000000,-0.552780,-0.493856,-0.567462,-0.351227,0.739484
414,-1.503319,1.321292,0.690758,-0.552780,1.312180,1.758018,-0.351227,-1.349059
415,0.872436,-0.755024,0.651184,-0.552780,-0.508183,-0.567462,-0.351227,0.739484
416,0.872436,-0.755024,0.000000,-0.552780,-0.493856,-0.567462,-0.351227,0.739484


In [40]:
x_train

,Pclass,Sex,Age,FamilySize,Fare,C,Q,S
0,0.826913,-0.737281,-0.592148,0.059127,-0.502163,-0.481772,-0.307390,0.618959
1,-1.565228,1.354813,0.638430,0.059127,0.786404,2.073341,-0.307390,-1.613803
2,0.826913,1.354813,-0.284503,-0.560660,-0.488580,-0.481772,-0.307390,0.618959
3,-1.565228,1.354813,0.407697,0.059127,0.420494,-0.481772,-0.307390,0.618959
4,0.826913,-0.737281,0.407697,-0.560660,-0.486064,-0.481772,-0.307390,0.618959
...,...,...,...,...,...,...,...,...
886,-0.369158,-0.737281,-0.207592,-0.560660,-0.386454,-0.481772,-0.307390,0.618959
887,-1.565228,1.354813,-0.822881,-0.560660,-0.044356,-0.481772,-0.307390,0.618959
888,0.826913,1.354813,0.000000,1.298700,-0.176164,-0.481772,-0.307390,0.618959
889,-1.565228,-0.737281,-0.284503,-0.560660,-0.044356,2.073341,-0.307390,-1.613803


In [52]:
t_train

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [42]:
Model = Sequential()
Model

<Sequential name=sequential_1, built=False>

In [43]:
Model.add(Dense(100, input_dim=8))

C:\Users\logihaal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [44]:
Model.add(Activation("relu"))

In [45]:
Model.add(Dense(2))

In [46]:
Model.add(Activation("softmax"))

In [47]:
Model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 100)            │           900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │           202 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 2)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,102 (4.30 KB)

 Trainable params: 1,102 (4.30 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
Model.compile(optimizer = "adam", loss = "categorical_crossentropy")

In [49]:
t_train.shape

(891,)

In [53]:
from keras.utils import to_categorical

# ラベルをワンホットエンコーディング
t_train = to_categorical(t_train, num_classes=2)


In [54]:
t_train.shape

(891, 2)

In [55]:
t_train

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]])

In [58]:
Model.fit(x_train,t_train,epochs=1000)

Epoch 1/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - loss: 0.4166
Epoch 2/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 704us/step - loss: 0.4043
Epoch 3/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 630us/step - loss: 0.4143
Epoch 4/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 593us/step - loss: 0.4170
Epoch 5/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 593us/step - loss: 0.3869
Epoch 6/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 593us/step - loss: 0.4226
Epoch 7/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 556us/step - loss: 0.4096
Epoch 8/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 519us/step - loss: 0.4187
Epoch 9/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 778us/step - loss: 0.4014
Epoch 10/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 630us/step - loss: 0.4146
Epoch 11/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 556us/step - loss: 0.3810
Epoch 12/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 556us/step - loss: 0.3977
Epoch 13/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 556us/step - loss: 0.4104
Epoch 14/1000
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 630us/step - loss: 0.4090
Epoch 15/1000
28/28 ━━━━━━━━━

In [60]:
t_test = Model.predict(x_test)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 616us/step


In [61]:
t_test = np.argmax(t_test, axis = 1)

In [62]:
t_test

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [77]:
x_test["Survived"] = t_test

In [79]:
x_test

,Pclass,Sex,Age,FamilySize,Fare,C,Q,S,survived,Survived
0,0.872436,-0.755024,0.334592,-0.552780,-0.497811,-0.567462,2.840354,-1.349059,0,0
1,0.872436,1.321292,1.323944,0.105516,-0.512660,-0.567462,-0.351227,0.739484,0,0
2,-0.315441,-0.755024,2.511166,-0.552780,-0.464532,-0.567462,2.840354,-1.349059,0,0
3,0.872436,-0.755024,-0.259019,-0.552780,-0.482888,-0.567462,-0.351227,0.739484,0,0
4,0.872436,1.321292,-0.654760,0.763813,-0.417971,-0.567462,-0.351227,0.739484,0,0
...,...,...,...,...,...,...,...,...,...,...
413,0.872436,-0.755024,0.000000,-0.552780,-0.493856,-0.567462,-0.351227,0.739484,0,0
414,-1.503319,1.321292,0.690758,-0.552780,1.312180,1.758018,-0.351227,-1.349059,1,1
415,0.872436,-0.755024,0.651184,-0.552780,-0.508183,-0.567462,-0.351227,0.739484,0,0
416,0.872436,-0.755024,0.000000,-0.552780,-0.493856,-0.567462,-0.351227,0.739484,0,0


In [80]:
t_output = x_test["Survived"]

In [81]:
t_output.index = t_output.index + 892

In [82]:
# インデックスの名前を'new_index_name'に変更
t_output.index.rename('PassengerId', inplace=True)

In [83]:
t_output

PassengerId
892     0
893     0
894     0
895     0
896     0
       ..
1305    0
1306    1
1307    0
1308    0
1309    0
Name: Survived, Length: 418, dtype: int64

In [84]:
# DataFrameをCSVに出力
t_output.to_csv("output.csv")